1. Import libs

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datetime

2. Download cifar10 data, +format

In [ ]:
cf10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cf10.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

3. Make a model- Maxpool every conv, Batchnormalize every conv; dropout% disabled - affects accuracy negatively. Remove second dense if needed

In [ ]:
model = tf.keras.models.Sequential([
    # Initial Conv Block
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Second Conv Block
    tf.keras.layers.Conv2D(128, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Third Conv Block
    tf.keras.layers.Conv2D(256, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Dense Layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])

4. Compile the model. 10 epochs for large dataset. 15+ has no accuracy increase.

In [ ]:

# compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# training parameters
EPOCHS = 50
BATCH_SIZE = 64

# callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3
)

# Train with data augmentation
history = model.fit(
    data_augmentation(x_train),
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(history)

5. Accuracy test

In [ ]:
eval = model.evaluate(x_test, y_test)
# print(eval)

6. Visualisation (commented out because it gives errors, don't know why)

In [ ]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
def predict_image(image_path):
    # Load and preprocess image
    image = tf.keras.preprocessing.image.load_img(
        image_path,
        target_size=(32, 32)
    )
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = image_array.astype('float32') / 255.0
    image_array = np.expand_dims(image_array, 0)

    # Make prediction
    predictions = model.predict(image_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = float(np.max(predictions[0]))

    # Display result
    plt.figure(figsize=(6, 6))
    plt.imshow(image_array[0])
    plt.title(f'Prediction: {predicted_class} ({confidence:.2%})')
    plt.axis('off')
    plt.show()

8. Predict user image

In [ ]:
image_path = 'deer1.jpg'
predict_image(image_path)

In [ ]:
# Save complete model (architecture + weights)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f'cifar10_cnn_model_{timestamp}.h5')



In [ ]:
# Function to load the model
def load_saved_model(model_path):
    """Load a saved model from path"""
    return tf.keras.models.load_model(model_path)

# Load the model
# loaded_model = load_saved_model('cifar10_cnn_model_20240315_120000')